In [1]:
from selenium import webdriver
from time import sleep
import time
from selenium.webdriver.common.by import By
import pandas as pd
from urllib.parse import urlparse
import shutil
import os

In [3]:
data = pd.read_csv("./data/Ahrefページ価値取得のコピー.csv", encoding="Shift-JIS")
data

,ドメイン,Unnamed: 1,Unnamed: 2
0,www.smauto.co.jp,NaN,NaN
1,www.orix.co.jp,NaN,NaN
2,www.ncsol.co.jp,NaN,NaN
3,t-mobility-s.co.jp,NaN,NaN
4,www.mitsubishi-autolease.com,NaN,NaN
...,...,...,...
217,http://www.rikuden-best.co.jp/,NaN,NaN
218,https://dcross.impress.co.jp/,NaN,NaN
219,https://rexev.co.jp/,NaN,NaN
220,https://www.amita-oshiete.jp/,NaN,NaN


In [4]:
USERNAME = "shotaro.yamazaki@flapnext.com"
PASSWORD = "zakizaki3939"


class AhrefsModel():

    def __init__(self,USERNAME,PASSWORD):
        self.username = USERNAME
        self.password = PASSWORD

    def mk_driver(self):
        options = webdriver.chrome.options.Options()
        # chrome_options = webdriver.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        # options.add_argument("-headless")
        options.add_argument("-no-sandbox")
        self.profile_path = './Profile ahrefs'
        options.add_argument('--user-data-dir=' + self.profile_path)
        UA = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15'
        options.add_argument('--user-agent=' + UA)

        driver = webdriver.Chrome(options=options)
        
        # driver = webdriver.Chrome(DRIVER_PATH,options=options)

        return driver


    def login_ahrefs(self,driver):
        target_url = "https://app.ahrefs.com/user/login"

        driver.get(target_url)
        sleep(5)

        username_input = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div[1]/div/div/div/div/form/div/div[1]/div/input')

        username_input.send_keys(self.username)
        sleep(1)

        password_input = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div[1]/div/div/div/div/form/div/div[2]/div/div/input')
        password_input.send_keys(self.password)
        sleep(1)

        login_burron = driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div/div[1]/div/div/div/div/form/div/button/div')
        login_burron.submit()
        sleep(5)

    def get_page_worth(self,data):
        
        df = pd.DataFrame()
        
        driver = self.mk_driver()
        self.login_ahrefs(driver)

        for i, domain in enumerate(data['ドメイン'].to_list()):

            domain_1 = urlparse(domain).netloc

            if domain_1:
                domain = domain_1

            i = i+1

            if i % 6 == 0:
                print('ドライバー　リセット！')
                driver.quit()
                shutil.rmtree(self.profile_path)
                os.mkdir(self.profile_path)
                driver = self.mk_driver()
                self.login_ahrefs(driver)

            print(domain)
            ahrefs_url = "https://app.ahrefs.com/positions-explorer/top-subfolders/subdomains/jp/all/all/all/all/all/all/all/1/traffic_desc?target="+domain+"%2F"

            driver.get(ahrefs_url)
            sleep(5)

            df_tmp = driver.find_element(By.XPATH,'//*[@id="main_se_data_table"]')

            html = df_tmp.get_attribute('outerHTML')
            df_tmp = pd.read_html(html)

            df_tmp = df_tmp[0].head(1)

            df_tmp = df_tmp[['トラフィック', '価値', 'キーワード']]
            df_tmp.insert(0,'ドメイン',domain)
            df_tmp.insert(4,'ahrefs上位ページ取得URL',ahrefs_url)

            df = pd.concat([df, df_tmp], join='outer')

        return df
        
    def get_page_worth_only_one(self,driver, url):

        df = pd.DataFrame()

        domain = urlparse(url).netloc

        if domain:
            url = domain

        print(url)
        ahrefs_url = "https://app.ahrefs.com/positions-explorer/top-subfolders/subdomains/jp/all/all/all/all/all/all/all/1/traffic_desc?target="+url+"%2F"

        driver.get(ahrefs_url)
        sleep(5)

        df_tmp = driver.find_element(By.XPATH,'//*[@id="main_se_data_table"]')

        html = df_tmp.get_attribute('outerHTML')
        df_tmp = pd.read_html(html)

        df_tmp = df_tmp[0].head(1)

        df_tmp = df_tmp[['トラフィック', '価値', 'キーワード']]
        df_tmp.insert(0,'ドメイン',url)
        df_tmp.insert(4,'ahrefs上位ページ取得URL',ahrefs_url)

        df = pd.concat([df, df_tmp], join='outer')
        driver.quit()

        return df


In [5]:
# am = AhrefsModel(USERNAME,PASSWORD)
# df = am.get_page_worth(data)
# driver = am.mk_driver()
# am.login_ahrefs(driver)

In [6]:
# driver.quit()

In [7]:
# data2 = pd.read_csv("./data/Ahrefページ価値取得のコピー.csv")
# data2

In [8]:
# am = AhrefsModel(USERNAME,PASSWORD)
# df2 = am.get_page_worth(data2)

In [9]:
# input_place_sec = driver.find_element(By.XPATH,'//*[@id="se_pe_target"]')
# input_place_sec.send_keys(domain)

# serch_button_sec = driver.find_element(By.XPATH,'//*[@id="se_pe_start_analysing"]')
# serch_button_sec.click()
# sleep(7)

In [10]:
# domains = ['qiita.com', 'www.maneo.jp/',"kodomomirai.com/"]

In [17]:
def get_page_worth_2(driver, data, df):
       for i, domain in enumerate(data['ドメイン'].to_list()):
              if i == 0:
                     input_place = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[1]/div[1]/div/div[1]/div[2]/div/div/div/input')
                     input_place.send_keys(domain)

                     serch_button = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[1]/div[1]/div/div[1]/button')
                     serch_button.click()
                     sleep(7)

                     top_subfolders = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[2]/div/div/div/div[3]/div[2]/div[3]/a')
                     top_subfolders.click()
                     sleep(10)
              else:
                     delete_button = driver.find_element(By.XPATH, '//*[@id="clear_se_pe_target"]')
                     delete_button.click()
                     
                     input_place_sec = driver.find_element(By.XPATH,'//*[@id="se_pe_target"]')
                     input_place_sec.send_keys(domain)

                     serch_button_sec = driver.find_element(By.XPATH,'//*[@id="se_pe_start_analysing"]')
                     serch_button_sec.click()
                     sleep(10)

              df_tmp = driver.find_element(By.XPATH,'//*[@id="main_se_data_table"]')

              html = df_tmp.get_attribute('outerHTML')
              df_tmp = pd.read_html(html)

              df_tmp = df_tmp[0].head(1)
              df_tmp.columns=['#', 'トラフィック', 'トラフィック.1', '価値', 'キーワード', 'ページ', 'パス',
                     'トップキーワード', '検索ボリューム', '順位', '順位..1']

              df_tmp = df_tmp[['トラフィック', '価値', 'キーワード', 'ページ']]

              ahrefs_url = driver.current_url

              df_tmp.insert(0,'ドメイン',domain)
              df_tmp.insert(5,'ahrefs上位ページ取得URL',ahrefs_url)

              df = pd.concat([df, df_tmp], join='outer')
       
       driver.quit()
       return df


In [15]:
am = AhrefsModel(USERNAME,PASSWORD)
driver = am.mk_driver()
am.login_ahrefs(driver)
sleep(3)
df2 = pd.DataFrame()

In [16]:
driver.quit()

In [25]:
data[data['ドメイン']=='https://www.tiger-inc.co.jp/']

,ドメイン
10,https://www.tiger-inc.co.jp/


In [51]:
data2 = data.loc[11:, :]
data2

,ドメイン
11,https://biz.moneyforward.com/
12,https://www.kaunet.com/
13,https://www.oal-net.co.jp/
14,http://www.chuden-al.co.jp/
15,https://www.ichinen.co.jp/
...,...
103,http://www.rikuden-best.co.jp/
104,https://dcross.impress.co.jp/
105,https://rexev.co.jp/
106,https://www.amita-oshiete.jp/


In [18]:
am = AhrefsModel(USERNAME,PASSWORD)
driver = am.mk_driver()
am.login_ahrefs(driver)
sleep(3)
df = pd.DataFrame()

df2_2 = get_page_worth_2(driver, data, df)

In [19]:
df2_2

,ドメイン,トラフィック,価値,キーワード,ページ,ahrefs上位ページ取得URL
0,www.smauto.co.jp,19491.0,"$4,754",12589,461,https://app.ahrefs.com/positions-explorer/top-...
0,www.orix.co.jp,350985.0,"$1,460,765",30410,3154,https://app.ahrefs.com/positions-explorer/top-...
0,www.ncsol.co.jp,3253.0,"$4,876",1467,189,https://app.ahrefs.com/positions-explorer/top-...
0,t-mobility-s.co.jp,16069.0,"$14,646",2023,88,https://app.ahrefs.com/positions-explorer/top-...
0,www.mitsubishi-autolease.com,1506.0,$851,1887,161,https://app.ahrefs.com/positions-explorer/top-...
...,...,...,...,...,...,...
0,http://www.rikuden-best.co.jp/,100.0,$21,159,15,https://app.ahrefs.com/positions-explorer/top-...
0,https://dcross.impress.co.jp/,9161.0,"$2,320",11153,2492,https://app.ahrefs.com/positions-explorer/top-...
0,https://rexev.co.jp/,501.0,$16,559,103,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.amita-oshiete.jp/,22757.0,"$7,968",9967,996,https://app.ahrefs.com/positions-explorer/top-...


In [55]:
df

,ドメイン,トラフィック,価値,キーワード,ahrefs上位ページ取得URL
0,https://dxhakusho.com/,868.0,$9,511,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.atled.jp/,79448.0,"$58,854",18412,https://app.ahrefs.com/positions-explorer/top-...
0,https://bonx.co.jp/,10346.0,"$6,606",2434,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.hitachi.co.jp/,394455.0,"$287,631",118157,https://app.ahrefs.com/positions-explorer/top-...
0,https://rpa-technologies.com/,12299.0,"$6,807",4228,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.kubota.co.jp/,232968.0,"$37,489",41546,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.kankyo-kanri.co.jp/,632.0,$157,886,https://app.ahrefs.com/positions-explorer/top-...
0,https://elaws.e-gov.go.jp/,1212705.0,"$570,649",245795,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.hrpro.co.jp/,511102.0,"$289,463",105404,https://app.ahrefs.com/positions-explorer/top-...
0,https://www.sn-hoki.co.jp/,78917.0,"$10,044",61436,https://app.ahrefs.com/positions-explorer/top-...


In [20]:
# df_new = pd.concat([df, df2_2])
df2_2.to_csv('./output/df3.csv', index=False)